In [1]:
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
import pandas as pd
import numpy as np
import cv2
import os
import glob

In [3]:
labels = pd.read_csv("G:\\Sorghum\\train_cultivar_mapping.csv")

In [4]:
labels

,image,cultivar
0,2017-06-16__12-24-20-930.png,PI_257599
1,2017-06-02__16-48-57-866.png,PI_154987
2,2017-06-12__13-18-07-707.png,PI_92270
3,2017-06-22__13-18-06-841.png,PI_152651
4,2017-06-26__12-56-48-642.png,PI_176766
...,...,...
22189,2017-06-16__12-27-16-266.png,PI_170787
22190,2017-06-28__11-19-57-454.png,PI_156393
22191,2017-06-28__10-20-32-417.png,PI_152923
22192,2017-06-28__12-47-02-714.png,PI_257599


In [5]:
class1 = labels.cultivar.unique()

In [6]:
class1

array(['PI_257599', 'PI_154987', 'PI_92270', 'PI_152651', 'PI_176766',
       'PI_156330', 'PI_329299', 'PI_52606', 'PI_145633', 'PI_273969',
       'PI_196586', 'PI_156463', 'PI_22913', 'PI_156393', 'PI_35038',
       'PI_152828', 'PI_152694', 'PI_329310', 'PI_251672', 'PI_154750',
       'PI_213900', 'PI_152816', 'PI_221651', 'PI_170787', 'PI_175919',
       'PI_297171', 'PI_329351', 'PI_152961', 'PI_179749', 'PI_196049',
       'PI_266927', 'PI_156871', 'PI_329300', 'PI_221548', 'PI_152966',
       'PI_273465', 'PI_146890', 'PI_152965', 'PI_329286', 'PI_180348',
       'PI_329338', 'PI_196583', 'PI_152771', 'PI_197542', 'PI_152967',
       'PI_156326', 'PI_154988', 'PI_145626', 'PI_329256', 'PI_157804',
       'PI_255744', 'PI_152860', 'PI_152730', 'PI_154844', 'PI_196598',
       'PI_329333', 'PI_152727', 'PI_145619', 'PI_297155', 'PI_156178',
       'PI_276837', 'PI_195754', 'PI_152751', 'PI_267573', 'PI_19770',
       'PI_152862', 'PI_155760', 'PI_152728', 'PI_297130', 'PI_229841

In [7]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(labels.cultivar)
labels['categories']=le.transform(labels.cultivar)

In [8]:
labels = labels[labels.categories !=100]

In [9]:
#from tensorflow.keras.preprocessing import image_dataset_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:


labels.head()

,image,cultivar,categories
0,2017-06-16__12-24-20-930.png,PI_257599,73
1,2017-06-02__16-48-57-866.png,PI_154987,29
2,2017-06-12__13-18-07-707.png,PI_92270,99
3,2017-06-22__13-18-06-841.png,PI_152651,6
4,2017-06-26__12-56-48-642.png,PI_176766,50


In [11]:
train_datagen = ImageDataGenerator(
    rotation_range = 0.9,
    zoom_range = 0.15,
    horizontal_flip = True,
    shear_range = 0.2,
    validation_split = 0.2

) 

train_data = train_datagen.flow_from_dataframe(
    dataframe = labels,
    directory =  "G:\\Sorghum\\train_images",
    x_col = 'image',
    y_col = 'cultivar',
    class_mode = 'sparse',
    target_size = (256,256),
    batch_size = 16,
    subset = 'training',
    shuffle = True

)

val_data = train_datagen.flow_from_dataframe(
    dataframe = labels,
    directory =  "G:\\Sorghum\\train_images",
    x_col = 'image',
    y_col = 'cultivar',
    class_mode = 'sparse',
    target_size = (256,256),
    batch_size = 16,
    subset = 'validation',
    shuffle = True
    
)


Found 17755 validated image filenames belonging to 100 classes.
Found 4438 validated image filenames belonging to 100 classes.


In [12]:
from tensorflow import keras
conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top = False,
    input_shape=(256,256,3)
)

In [13]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [14]:
inputs = keras.Input(shape = (256 ,256 , 3))
x = keras.applications.vgg16.preprocess_input(inputs)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.2)(x)

outputs = layers.Dense(100,activation = "softmax")(x)
model = keras.Model(inputs,outputs)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "rmsprop",metrics=["accuracy"])

In [ ]:
history = model.fit(train_data,epochs = 50, validation_data = val_data)

Epoch 1/50
1110/1110 [==============================] - 1215s 1s/step - loss: 170.1151 - accuracy: 0.0101 - val_loss: 4.5930 - val_accuracy: 0.0131
Epoch 2/50
1110/1110 [==============================] - 1116s 1s/step - loss: 4.5942 - accuracy: 0.0125 - val_loss: 4.5960 - val_accuracy: 0.0131
Epoch 3/50
1110/1110 [==============================] - 1107s 997ms/step - loss: 4.5935 - accuracy: 0.0139 - val_loss: 4.5971 - val_accuracy: 0.0131
Epoch 4/50
1110/1110 [==============================] - 1126s 1s/step - loss: 15.1930 - accuracy: 0.0123 - val_loss: 4.5966 - val_accuracy: 0.0131
Epoch 5/50
1110/1110 [==============================] - 1138s 1s/step - loss: 5.1023 - accuracy: 0.0129 - val_loss: 4.5952 - val_accuracy: 0.0131
Epoch 6/50
1110/1110 [==============================] - 1082s 975ms/step - loss: 4.5935 - accuracy: 0.0124 - val_loss: 4.5937 - val_accuracy: 0.0131
Epoch 7/50
1110/1110 [==============================] - 1101s 992ms/step - loss: 4.5933 - accuracy: 0.0118 - val_lo

In [18]:
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath = "imagenet_model.keras", 
        save_best_only = True,
        monitor = "val_loss"
    )
    


In [19]:
from keras.callbacks import EarlyStopping
ES = EarlyStopping(monitor = "val_loss" , patience = 5)
history = model.fit(train_data,epochs = 50, validation_data = val_data, callbacks = [checkpoint, ES])

Epoch 1/50
1110/1110 [==============================] - 880s 792ms/step - loss: 4.5979 - accuracy: 0.0124 - val_loss: 4.6009 - val_accuracy: 0.0092
Epoch 2/50
1110/1110 [==============================] - 877s 790ms/step - loss: 4.5979 - accuracy: 0.0122 - val_loss: 4.5965 - val_accuracy: 0.0131
Epoch 3/50
1110/1110 [==============================] - 804s 725ms/step - loss: 4.5983 - accuracy: 0.0126 - val_loss: 4.6003 - val_accuracy: 0.0092
Epoch 4/50
1110/1110 [==============================] - 793s 714ms/step - loss: 4.5984 - accuracy: 0.0110 - val_loss: 4.5986 - val_accuracy: 0.0135
Epoch 5/50
1110/1110 [==============================] - 792s 714ms/step - loss: 4.5983 - accuracy: 0.0133 - val_loss: 4.5976 - val_accuracy: 0.0115
Epoch 6/50
1110/1110 [==============================] - 825s 743ms/step - loss: 4.5985 - accuracy: 0.0133 - val_loss: 4.5991 - val_accuracy: 0.0131
Epoch 7/50
1110/1110 [==============================] - 808s 728ms/step - loss: 4.5978 - accuracy: 0.0115 - val_

In [20]:

inputs = keras.Input(shape = (256 , 256 , 3))

x = layers.Rescaling(1.0 /255)(inputs)

x = layers.Conv2D(filters = 32 , kernel_size = 3 , activation = 'relu')(x)
x = layers.MaxPooling2D(pool_size = 2)(x)
x = layers.Conv2D(filters = 64 , kernel_size = 3 , activation = 'relu')(x)
x = layers.MaxPooling2D(pool_size = 2)(x)
x = layers.Conv2D(filters = 128 , kernel_size = 3 , activation = 'relu')(x)
x = layers.MaxPooling2D(pool_size = 2)(x)
x = layers.Conv2D(filters = 256 , kernel_size = 3 , activation = 'relu')(x)
x = layers.MaxPooling2D(pool_size = 2)(x)
x = layers.Conv2D(filters = 256 , kernel_size = 3 , activation = 'relu')(x)

x = layers.Flatten()(x)
output = layers.Dense(100 ,activation = 'softmax') (x)

In [21]:
model = keras.Model (inputs = inputs , outputs = output)

In [22]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                       

In [23]:
model.compile( loss = "binary_crossentropy" , optimizer = "rmsprop" , metrics =['accuracy'])

In [24]:
train_data

In [25]:
from numpy.lib.npyio import save
callbacks = [

keras.callbacks.ModelCheckpoint(filepath = "catvsdgoscnn1.keras" ,save_best_only = True, monitor ="val_loss"),
keras.callbacks.EarlyStopping            
             
]

In [26]:
history = model.fit(train_data, epochs= 30 ,validation_data = val_data , callbacks = callbacks)

TypeError: set_model() missing 1 required positional argument: 'model'